# RLHF

Reinforcement Learning from Human Feedback

* Initial LLM
* Train a Reward Model(RM)
* Optimize(fine-tune)LLM using RL(e.g. PPO) based on trained RM


## Reward Model

This is a separate model which is calibrated with human preferences. The model predicts rewards for LLM input-outputs.

The model is used by RL to fine-tune LLM.

* Collect samples(inputs and outputs) from pre-trained LLM
* Human annotatatates samples that used in scoring and ranking
* Build training dataset as sample-reward pair
* Train Reward Model that learns from human preferences on LLM outputs

## TRL 

Transformer Reinforcement Learning library accommodates several RL approaches to fine-tune transformer-based LLMs.

Proximal Policy Optimization (PPO)  optimizes LLM using <prompt, reponse, reward > triplets.

In [ ]:
from trl import PPOTrainer, PPOConfig, create_reference_model, AutoModelForCausalLMWithValueHead
from trl.core import respons_to_batch

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")

# refrence of loaded pre-trained model before optimization
model_ref = create_reference_model(model)

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
prompt = 'My plan for today was '
sample_text = tokenizer.encode(query_txt, return_tensor='pt')
# respons_to_batch is same as model.generate() in RL library
response = respons_to_batch(model, sample_text)

ppo_config = PPOConfig(batcch_size=1)
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)
reward = [torch.tensor(1.0)]
train_stats = ppo_trainer.step([sample_text[0]], [response[0]], reward)